In [1]:
from seapodym_lmtl_python.config.parameters import (
    FunctionalGroupUnit,
    FunctionalGroups,
    FunctionParameters,
    PathParametersUnit,
    PathParameters,
    Parameters
)

from seapodym_lmtl_python.config.client import (
    init_client_locally,
    close_client_locally
)

from seapodym_lmtl_python.config.parser import parse_configuration_file

from seapodym_lmtl_python.config.saving import (
    save_configuration_locally,
    save_outputs_locally
)

from seapodym_lmtl_python.config import model_configuration
from seapodym_lmtl_python.process import biomass, pre_production, production

### The model is run in 8 steps:
1. Parse the configuration file.
2. Initialize the Dask client/cluster.
3. Generate the configuration.
4. Run the pre-production process.
(4.bis Save the configuration.)
5. Run the production process.
(5.bis Save the configuration.)
6. Run the biomass process.
7. Save the outputs.
8. Close the client.

---
# 1. PARSE (CLI integration is done here)

In [2]:
# parameters = parse_configuration_file(configuration_file_path, kwargs)

> We are doing it by hand for now. ✍🏻

In [3]:
path_forcing = "/Users/ash/Documents/Workspaces/Data/phd/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1709010880580.nc"
path_bio = "/Users/ash/Documents/Workspaces/Data/phd/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1709007157635.nc"

In [4]:
f_param = FunctionParameters(
    inv_lambda_max=2109,
    inv_lambda_rate=0.125,
    temperature_recruitment_rate=0.125,
    temperature_recruitment_max=527,
    energy_transfert=0.0042,
)
# ---------------------------- #
f_groups = []
for dl, nl, ec in zip(
    [1, 2, 2, 3, 3, 3], [1, 2, 1, 3, 2, 1], [0.17, 0.1, 0.22, 0.18, 0.13, 0.2]
):
    f_groups.append(
        FunctionalGroupUnit(
            name=f"D{dl}N{nl}",
            day_layer=dl,
            night_layer=nl,
            energy_coefficient=ec,
        )
    )
f_groups = FunctionalGroups(functional_groups=f_groups)
# ---------------------------- #
p_param = PathParameters(
    temperature=PathParametersUnit(forcing_path=path_forcing, name="T"),
    primary_production=PathParametersUnit(forcing_path=path_bio, name="npp"),
)
# ---------------------------- #
parameters = Parameters(
    function_parameters=f_param,
    functional_groups_parameters=f_groups,
    path_parameters=p_param,
)
# ---------------------------- #

---
# 2. INITIALIZE THE DASK CLIENT/CLUSTER

In [5]:
client = init_client_locally(None)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54432,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:54444,Total threads: 2
Dashboard: http://127.0.0.1:54446/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:54435,


---
# 3. GENERATE THE CONFIGURATION

In [6]:
configuration = model_configuration.process(parameters)
configuration

<xarray.Dataset> Size: 44MB
Dimensions:                              (latitude: 24, longitude: 24,
                                          time: 4748, depth: 3,
                                          functional_group: 6)
Coordinates:
  * latitude                             (latitude) float32 96B 22.08 ... 24.0
  * longitude                            (longitude) float32 96B -159.0 ... -...
  * time                                 (time) datetime64[ns] 38kB 2010-01-0...
  * depth                                (depth) float32 12B 1.0 2.0 3.0
  * functional_group                     (functional_group) int64 48B 0 1 ... 5
Data variables:
    primary_production                   (time, latitude, longitude) float32 11MB ...
    temperature                          (time, depth, latitude, longitude) float32 33MB ...
    inv_lambda_max                       float64 8B 2.109e+03
    inv_lambda_rate                      float64 8B 0.125
    temperature_recruitment_rate         float64 8B 0.125
    temperature_recruitment_max          float64 8B 527.0
    energy_transfert                     float64 8B 0.0042
    day_position                         (functional_group) int64 48B 1 2 ... 3
    night_position                       (functional_group) int64 48B 1 2 ... 1
    functional_group_energy_coefficient  (functional_group) float64 48B 0.17 ...

# TODO
## Ajouter les parametres de day et night layers.

---
# 1. PARSE (CLI integration is done here)

---
# 1. PARSE (CLI integration is done here)

In [ ]:
# 4. RUN THE PRE-PRODUCTION PROCESS
configuration = pre_production.process(client, configuration)

# 4.bis SAVE THE CONFIGURATION ?
save_configuration_locally(configuration)

# 5. RUN THE PRODUCTION PROCESS

configuration = production.process(configuration)

# 5.bis SAVE THE CONFIGURATION ?
save_configuration_locally(configuration)

# 6. RUN BIOMASS PROCESS
configuration = biomass.process(client, configuration)

# 7. SAVE THE OUTPUTS
save_outputs_locally(configuration)

# 8. CLOSE THE CLIENT
close_client_locally(client)